In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras import layers
import os
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from tabulate import tabulate


In [3]:
#df1Back = pd.read_csv('/content/drive/MyDrive/MovePort/1/back/cop_1.csv')
#df1Back.head(15)
#data_path = '/content/drive/MyDrive/MovePort/data'
data_path = '/content/drive/MyDrive/MovePort/'
Modified_Df_PATH = '/content/drive/MyDrive/MovePortModif/'
participants = sorted([p for p in os.listdir(data_path) if p.isdigit()], key=int)
print("Participants disponibles :", participants)

Participants disponibles : ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']


In [4]:
df1Back = pd.read_csv('/content/drive/MyDrive/MovePort/1/back/emg_1.csv')
df1Back.head(6)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,132430,132431,132432,132433,132434,132435,132436,132437,132438,132439
0,R_Vlat,-0.704956,2.014160,-0.704956,-0.704956,-0.704956,-6.848145,-4.531860,-4.733276,1.812744,...,-4.733276,1.409912,8.358765,-0.100708,0.906372,-3.524780,-2.719116,-9.365845,-6.546021,-3.121948
1,R_RF,1.309204,1.309204,1.309204,1.309204,1.309204,1.309204,1.309204,1.309204,1.309204,...,1.309204,-0.201416,2.114868,6.948853,6.948853,-1.107788,0.402832,-8.963013,-3.625488,-11.480713
2,R_ST,-2.316284,-2.316284,-2.316284,-2.316284,-2.316284,-2.316284,-2.316284,-2.316284,-2.316284,...,0.201416,-2.416992,1.812744,-0.906372,-0.805664,-6.747437,-2.215576,-1.107788,-6.042480,-3.323364
3,R_TA,-20.242310,-16.415405,-16.314697,-12.387085,-11.581421,-13.494873,-3.121948,-5.740356,-4.431152,...,-3.726196,-35.147095,-29.507446,-7.553101,5.136108,18.832397,7.754517,-20.544434,-78.451538,-124.374390
4,L_Vlat,-6.747437,-1.208496,-1.107788,-2.316284,1.208496,-0.906372,0.100708,-0.302124,-1.510620,...,2.819824,1.913452,11.077881,26.687622,28.198242,32.931519,27.593994,30.010986,29.306028,24.371338
5,L_RF,2.920532,2.618408,1.107788,-1.913452,-2.618408,-4.129028,-1.309204,1.510620,-2.719116,...,-16.516113,-20.846558,-18.630981,-17.019653,-7.049561,4.431152,10.775757,23.666382,27.996826,33.636475


In [10]:
df1Back = pd.read_csv('/content/drive/MyDrive/MovePort/1/back/emg_1.csv')
df1Back = df1Back.transpose()
df1Back.head()
#df1Back.info()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Unnamed: 0,R_Vlat,R_RF,R_ST,R_TA,L_Vlat,L_RF,L_ST,L_TA,R_MG,R_LG,R_SOL,R_IL,L_MG,L_LG,L_SOL,L_IL
0,-0.704956,1.309204,-2.316284,-20.24231,-6.747437,2.920532,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478
1,2.01416,1.309204,-2.316284,-16.415405,-1.208496,2.618408,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478
2,-0.704956,1.309204,-2.316284,-16.314697,-1.107788,1.107788,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478
3,-0.704956,1.309204,-2.316284,-12.387085,-2.316284,-1.913452,-1.208496,-6.948853,-0.906372,-12.387085,54.180908,-2.5177,-0.704956,10.876465,-3.52478,-3.52478


In [28]:
activities = ["back", "forward", "halfsquat", "still"]
modalities = ["emg_1", "imu_1", "ips_1"]
# Fonction pour charger et traiter un fichier CSV
def process_file(file_path):
    if os.path.exists(file_path):
        # Charger les données
        data = pd.read_csv(file_path)
        # Supprimer les valeurs nulles
        data = data.dropna()
        # Transposer les données
        return data.T
    else:
        print(f"Fichier non trouvé : {file_path}")
        return None

In [ ]:
group_size = 5

# Fonction pour créer un dataset pour un groupe de participants
def create_group_dataset(group_id, participants):
    group_data = {activity: {modality: [] for modality in modalities} for activity in activities}

    # Parcourir les participants du groupe
    for participant_id in participants:
        participant_path = os.path.join(data_path, str(participant_id))
        # Parcourir les activités
        for activity in activities:
            activity_path = os.path.join(participant_path, activity)
            # Parcourir les modalités
            for modality in modalities:
                file_name = f"{modality}_1.csv"
                file_path = os.path.join(activity_path, file_name)
                # Charger et traiter le fichier
                data = process_file(file_path)
                if data is not None:
                    group_data[activity][modality].append(data)
    # Fusionner les données par activité et modalité
    merged_data = {}
    for activity in activities:
        merged_data[activity] = {}
        for modality in modalities:
            # Concaténer les données de tous les participants pour une activité et modalité données
            merged_data[activity][modality] = pd.concat(group_data[activity][modality], axis=1)

    # Sauvegarder les fichiers fusionnés
    output_dir = f"/content/drive/MyDrive/MovePort/group_{group_id}"
    os.makedirs(output_dir, exist_ok=True)

    for activity in activities:
        for modality in modalities:
            output_file = os.path.join(output_dir, f"{activity}_{modality}.csv")
            merged_data[activity][modality].to_csv(output_file, index=False)
            print(f"Données sauvegardées : {output_file}")


In [6]:

"""for participant in participants:
    participant_path = os.path.join(data_path, participant)
    print(f"Participant: {participant}")

    # Filtrer les éléments non désirés
    activities = [a for a in os.listdir(participant_path) if not a.startswith('.')]
    for activity in activities:
        activity_path = os.path.join(participant_path, activity)

        # Vérifier si c'est un dossier avant d'essayer d'en lister le contenu
        if not os.path.isdir(activity_path):
            continue

        files = os.listdir(activity_path)  # Liste des fichiers CSV
        print(f"  Activity: {activity}, Files: {files}")"""

'for participant in participants:\n    participant_path = os.path.join(data_path, participant)\n    print(f"Participant: {participant}")\n\n    # Filtrer les éléments non désirés\n    activities = [a for a in os.listdir(participant_path) if not a.startswith(\'.\')]\n    for activity in activities:\n        activity_path = os.path.join(participant_path, activity)\n\n        # Vérifier si c\'est un dossier avant d\'essayer d\'en lister le contenu\n        if not os.path.isdir(activity_path):\n            continue\n\n        files = os.listdir(activity_path)  # Liste des fichiers CSV\n        print(f"  Activity: {activity}, Files: {files}")'

In [14]:
from itertools import islice

batch_size = 5

def batch_iterable(iterable, size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, size)):
        yield batch

participants_sorted = sorted(participants, key=lambda x: int(x))

# Parcours par lot
for batch_number, participant_batch in enumerate(batch_iterable(participants_sorted, batch_size), start=1):
    print(f"Traitement du lot {batch_number} : {participant_batch}")

    for participant in participant_batch:
        participant_path = os.path.join(data_path, participant)
        print(f"Participant: {participant}")

        # Filtrer les activités
        activities = [a for a in os.listdir(participant_path) if not a.startswith('.') and os.path.isdir(os.path.join(participant_path, a))]

        for activity in activities:
            activity_path = os.path.join(participant_path, activity)
            files = [f for f in os.listdir(activity_path) if f.endswith('.csv') and not f.startswith('.')]

            # Filtrer les fichiers spécifiques avec le format exact
            emg_files = [f for f in files if f.startswith('emg_1') and f.endswith('.csv')]
            imu_files = [f for f in files if f.startswith('imu_1') and f.endswith('.csv')]
            ips_files = [f for f in files if f.startswith('ips_1') and f.endswith('.csv')]
            all_files = emg_files + imu_files + ips_files
            #emg_filesT = pd.read_csv(activity_path + "/" + emg_files[0])
            #emg_filesT.head(5)
            print(f"  Activity: {activity}")
            #print(f"    EMG Files: {emg_files}")
            #print(f"    IMU Files: {imu_files}")
            #print(f"    IPS Files: {ips_files}")
            print(f"    All Files: {all_files}")

    print(f"Fin du traitement du lot {batch_number}\n")

Traitement du lot 1 : ['1', '2', '3', '4', '5']
Participant: 1
  Activity: forward
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: still
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: back
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: halfsquat
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
Participant: 2
  Activity: back
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: forward
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: halfsquat
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: still
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
Participant: 3
  Activity: back
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: forward
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: still
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
  Activity: halfsquat
    All Files: ['emg_1.csv', 'imu_1.csv', 'ips_1.csv']
Participant: 4
 

In [15]:
batch_size = 5  # Taille du lot

# Fonction pour créer un itérable par lot
def batch_iterable(iterable, size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, size)):
        yield batch

# Fonction pour traiter un participant et ses fichiers CSV
def process_participant_files(participant_path, activities):
    processed_data = {}

    for activity in activities:
        activity_path = os.path.join(participant_path, activity)
        files = [f for f in os.listdir(activity_path) if f.endswith('.csv') and not f.startswith('.')]  # Filtrer uniquement les fichiers CSV

        # Filtrer les fichiers spécifiques avec le format exact
        emg_files = [f for f in files if f.startswith('emg_1') and f.endswith('.csv')]
        imu_files = [f for f in files if f.startswith('imu_1') and f.endswith('.csv')]
        ips_files = [f for f in files if f.startswith('ips_1') and f.endswith('.csv')]

        print(f"  Activity: {activity}")
        print(f"    EMG Files: {emg_files}")
        print(f"    IMU Files: {imu_files}")
        print(f"    IPS Files: {ips_files}")

        # Dictionnaire pour stocker les DataFrames traités pour cette activité
        processed_data[activity] = {}
        # Traiter les fichiers de chaque groupe (EMG, IMU, IPS)
        for file_group, file_list in [("EMG", emg_files), ("IMU", imu_files), ("IPS", ips_files)]:
            for file in file_list:
                file_path = os.path.join(activity_path, file)
                print(f"      {file_group} File: {file}")
                try:
                    # Lire le fichier CSV dans un DataFrame
                    df = pd.read_csv(file_path)
                    # Transposer (pivoter) le DataFrame
                    df_transposed = df.transpose()
                    print(f"      Transposed DataFrame for {file_group} - {file}:")

                    # Ajouter le DataFrame transposé au dictionnaire des données traitées
                    processed_data[activity][file_group] = df_transposed

                except Exception as e:
                    print(f"        Impossible de lire ou de transposer le fichier {file}: {e}")

    return processed_data


# Parcours par lot des participants
def process_data_in_batches(data_path, batch_size=5):
    # Générer la liste des participants de 1 à 25 (ou plus si nécessaire)
    participants = [str(i) for i in range(1, 26)]  # Ici on génère les participants de '1' à '25'

    for batch_number, participant_batch in enumerate(batch_iterable(participants, batch_size), start=1):
        print(f"Traitement du lot {batch_number} : {participant_batch}")

        for participant in participant_batch:
            participant_path = os.path.join(data_path, participant)
            print(f"Participant: {participant}")

            # Filtrer les activités
            activities = [a for a in os.listdir(participant_path) if not a.startswith('.') and os.path.isdir(os.path.join(participant_path, a))]

            # Traiter les fichiers de chaque participant et activité
            processed_data = process_participant_files(participant_path, activities)

            # Vous pouvez accéder aux données traitées et les utiliser pour l'alignement temporel et la synchronisation
            print(f"  Données traitées pour le participant {participant}:")

        print(f"Fin du traitement du lot {batch_number}\n")


# Exemple d'appel de la fonction principale
process_data_in_batches(data_path)

Traitement du lot 1 : ['1', '2', '3', '4', '5']
Participant: 1
  Activity: forward
    EMG Files: ['emg_1.csv']
    IMU Files: ['imu_1.csv']
    IPS Files: ['ips_1.csv']
      EMG File: emg_1.csv
      Transposed DataFrame for EMG - emg_1.csv:
      IMU File: imu_1.csv
      Transposed DataFrame for IMU - imu_1.csv:
      IPS File: ips_1.csv
      Transposed DataFrame for IPS - ips_1.csv:
  Activity: still
    EMG Files: ['emg_1.csv']
    IMU Files: ['imu_1.csv']
    IPS Files: ['ips_1.csv']
      EMG File: emg_1.csv
      Transposed DataFrame for EMG - emg_1.csv:
      IMU File: imu_1.csv
      Transposed DataFrame for IMU - imu_1.csv:
      IPS File: ips_1.csv
      Transposed DataFrame for IPS - ips_1.csv:
  Activity: back
    EMG Files: ['emg_1.csv']
    IMU Files: ['imu_1.csv']
    IPS Files: ['ips_1.csv']
      EMG File: emg_1.csv
      Transposed DataFrame for EMG - emg_1.csv:
      IMU File: imu_1.csv
      Transposed DataFrame for IMU - imu_1.csv:
      IPS File: ips_1.csv
   

In [17]:
import os
import pandas as pd
from itertools import islice

def batch_iterable(iterable, size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, size)):
        yield batch

def process_participant_files(participant_path, activities):
    processed_data = {}

    for activity in activities:
        activity_path = os.path.join(participant_path, activity)
        files = [f for f in os.listdir(activity_path) if f.endswith('.csv') and not f.startswith('.')]  # Filtrer uniquement les fichiers CSV

        # Filtrer les fichiers spécifiques avec le format exact
        emg_files = [f for f in files if f.startswith('emg_1') and f.endswith('.csv')]
        imu_files = [f for f in files if f.startswith('imu_1') and f.endswith('.csv')]
        ips_files = [f for f in files if f.startswith('ips_1') and f.endswith('.csv')]
        print(f"  Activity: {activity}")
        print(f"    EMG Files: {emg_files}")
        print(f"    IMU Files: {imu_files}")
        print(f"    IPS Files: {ips_files}")
        processed_data[activity] = {}

        for file_group, file_list in [("EMG", emg_files), ("IMU", imu_files), ("IPS", ips_files)]:
            for file in file_list:
                file_path = os.path.join(activity_path, file)
                print(f"      {file_group} File: {file}")
                try:
                    df = pd.read_csv(file_path)
                    df_transposed = df.transpose()
                    print(f"      Transposed DataFrame for {file_group} - {file}:")
                    processed_data[activity][file_group] = df_transposed
                except Exception as e:
                    print(f"        Impossible de lire ou de transposer le fichier {file}: {e}")
    return processed_data


def process_data_in_batches(data_path, participants_list, batch_size=5):
    for batch_number, participant_batch in enumerate(batch_iterable(participants_list, batch_size), start=1):
        print(f"Traitement du lot {batch_number} : {participant_batch}")
        for participant in participant_batch:
            participant_path = os.path.join(data_path, str(participant))
            print(f"Participant: {participant}")
            # Filtrer les activités
            activities = [a for a in os.listdir(participant_path) if not a.startswith('.') and os.path.isdir(os.path.join(participant_path, a))]
            processed_data = process_participant_files(participant_path, activities)
            # Vous pouvez accéder aux données traitées et les utiliser pour l'alignement temporel et la synchronisation
            print(f"  Données traitées pour le participant {participant}:")
        print(f"Fin du traitement du lot {batch_number}\n")
    return processed_data


participants_to_process = [1, 2, 3, 4, 5]
processed_data = process_data_in_batches(data_path, participants_list=participants_to_process, batch_size=5)
processed_data

Traitement du lot 1 : [1, 2, 3, 4, 5]
Participant: 1
  Activity: forward
    EMG Files: ['emg_1.csv']
    IMU Files: ['imu_1.csv']
    IPS Files: ['ips_1.csv']
      EMG File: emg_1.csv
      Transposed DataFrame for EMG - emg_1.csv:
      IMU File: imu_1.csv
      Transposed DataFrame for IMU - imu_1.csv:
      IPS File: ips_1.csv
      Transposed DataFrame for IPS - ips_1.csv:
  Activity: still
    EMG Files: ['emg_1.csv']
    IMU Files: ['imu_1.csv']
    IPS Files: ['ips_1.csv']
      EMG File: emg_1.csv
      Transposed DataFrame for EMG - emg_1.csv:
      IMU File: imu_1.csv
      Transposed DataFrame for IMU - imu_1.csv:
      IPS File: ips_1.csv
      Transposed DataFrame for IPS - ips_1.csv:
  Activity: back
    EMG Files: ['emg_1.csv']
    IMU Files: ['imu_1.csv']
    IPS Files: ['ips_1.csv']
      EMG File: emg_1.csv
      Transposed DataFrame for EMG - emg_1.csv:
      IMU File: imu_1.csv
      Transposed DataFrame for IMU - imu_1.csv:
      IPS File: ips_1.csv
      Transpo

{'forward': {'EMG':                   0         1          2         3         4         5   \
  Unnamed: 0    R_Vlat      R_RF       R_ST      R_TA    L_Vlat      L_RF   
  0          -5.337524 -4.733276   4.330444   0.50354 -2.618408  1.611328   
  1          -5.740356 -7.553101  -5.136108 -1.812744 -2.618408  4.733276   
  2          -6.344604 -6.948853  -9.265137   4.02832 -2.618408  1.712036   
  3          -2.719116 -3.121948 -15.408324 -2.114868 -2.618408 -1.712036   
  ...              ...       ...        ...       ...       ...       ...   
  112755       0.50354  -0.50354 -11.984253 -4.632568  6.445312 -5.740356   
  112756       1.00708  1.107788 -32.427979 -4.330444  2.719116 -3.927612   
  112757     -4.129028 -0.906372 -34.643555 -5.639648  5.236816 -5.236816   
  112758      2.316284 -2.920532  -23.26355  1.409912  4.934692 -7.553101   
  112759       3.02124 -3.424072 -14.804077 -2.114868  5.337524 -3.726196   
  
                     6         7          8           9

In [25]:
processed_data['forward']


{'EMG':                   0         1          2         3         4         5   \
 Unnamed: 0    R_Vlat      R_RF       R_ST      R_TA    L_Vlat      L_RF   
 0          -5.337524 -4.733276   4.330444   0.50354 -2.618408  1.611328   
 1          -5.740356 -7.553101  -5.136108 -1.812744 -2.618408  4.733276   
 2          -6.344604 -6.948853  -9.265137   4.02832 -2.618408  1.712036   
 3          -2.719116 -3.121948 -15.408324 -2.114868 -2.618408 -1.712036   
 ...              ...       ...        ...       ...       ...       ...   
 112755       0.50354  -0.50354 -11.984253 -4.632568  6.445312 -5.740356   
 112756       1.00708  1.107788 -32.427979 -4.330444  2.719116 -3.927612   
 112757     -4.129028 -0.906372 -34.643555 -5.639648  5.236816 -5.236816   
 112758      2.316284 -2.920532  -23.26355  1.409912  4.934692 -7.553101   
 112759       3.02124 -3.424072 -14.804077 -2.114868  5.337524 -3.726196   
 
                    6         7          8           9          10        11  \